In [17]:
import sympy as sp
from sympy.physics.quantum import TensorProduct, Dagger
from sympy.physics.quantum.trace import Tr

def half_wave_plate_sympy(theta):
    c = sp.cos(2 * theta)
    s = sp.sin(2 * theta)
    HWP_matrix = sp.Matrix([[c, s], [s, -c]])
    return HWP_matrix

# Define theta as a symbolic variable and create operator
theta_1, theta_2 = sp.symbols('theta_1 theta_2', real=True)
HWP_operator_1 = half_wave_plate_sympy(theta_1)
HWP_operator_2 = half_wave_plate_sympy(theta_2)

# Define horizontal and vertical polarization states
H = sp.Matrix([1, 0])
V = sp.Matrix([0, 1])

# Bell state (vector)
Phi_plus = (TensorProduct(H, H) + TensorProduct(V, V)) / sp.sqrt(2)
Phi_minus = (TensorProduct(H, H) - TensorProduct(V, V)) / sp.sqrt(2)
Psi_plus = (TensorProduct(H, V) + TensorProduct(V, H)) / sp.sqrt(2)
Psi_minus = (TensorProduct(H, V) - TensorProduct(V, H)) / sp.sqrt(2)


# Apply HWP to the first qubit of the entangled state
HWP_total_operator = TensorProduct(HWP_operator_1, HWP_operator_2)
after_HWP_state = HWP_total_operator * Phi_plus
after_HWP_state.subs({theta_1: 0, theta_2: 0})


Matrix([
[sqrt(2)/2],
[        0],
[        0],
[sqrt(2)/2]])

In [11]:
# Beam cube implementation:

# Define the projection operator for vertically & horizontally polarized light
P_V = V * V.T  # |V><V| projection operator
P_H = H * H.T  # |H><H| projection operator

# Projects into each combination of 2 output ports from either beam line
VH_operator = TensorProduct(P_V, P_H)
VV_operator = TensorProduct(P_V, P_V)
HH_operator = TensorProduct(P_H, P_H)
HV_operator = TensorProduct(P_H, P_V)

after_HWP_state.subs({theta_1: 0, theta_2: 0})

# Probability of getting coincidence in HV port
(after_HWP_state.T * HV_operator * after_HWP_state).subs({theta_1:sp.pi/4, theta_2:0})[0]

0

### Density Matrix Formalism

In [12]:
rho_phi_plus = TensorProduct(Phi_plus, Dagger(Phi_plus))

rho_after_HWP = HWP_total_operator * rho_phi_plus * Dagger(HWP_total_operator)

probability_HV = (Dagger(TensorProduct(H,V)) * rho_after_HWP * TensorProduct(H,V))[0]
probability_HV.subs({theta_1:sp.pi/4, theta_2:0})

1/2

### Reduced density matrices (for either Alice or Bob)

In [16]:
def partial_trace(rho, trace_out, basis0, basis1):

    if trace_out == 0:
        operator_B0 = TensorProduct(sp.eye(2), basis0)
        operator_B1 = TensorProduct(sp.eye(2), basis1)
    elif trace_out == 1:
        operator_B0 = TensorProduct(basis0, sp.eye(2))
        operator_B1 = TensorProduct(basis1, sp.eye(2))

    return Dagger(operator_B0) * rho * operator_B0 + Dagger(operator_B1) * rho * operator_B1 

rho_Alice_after_HWP = partial_trace(rho_after_HWP, trace_out=1, basis0=H, basis1=V)
rho_Bob_after_HWP = partial_trace(rho_after_HWP, trace_out=0, basis0=H, basis1=V)

# probability alice measures H
p_H_Alice = (Dagger(H) * rho_Alice_after_HWP * H)
p_H_Bob = (Dagger(H) * rho_Alice_after_HWP * H)

rA = sp.simplify(p_H_Alice)[0,0]
rB = sp.simplify(p_H_Bob)[0,0]

print(f"Alice: {rA}")
print(f"Bob: {rB}") 

Alice: 1/2
Bob: 1/2


so we see that alice and bob always have a 50/50 chance of measuring H or V no matter what setting the HWP are set to